# Obtenção dos Dados:

Este projeto foi inspirado pelas atividades previamente realizadas por [Kizzy Terra](#https://github.com/kterra) e os dados utilizados estão neste [repositório](#https://github.com/kterra/ciencia-de-dados-aplicada), que segundo a própria autora foram dados obtidos pelo PRODES.

_"O projeto PRODES realiza o monitoramento por satélites do desmatamento por corte raso na Amazônia Legal e produz, desde 1988, as taxas anuais de desmatamento na região, que são usadas pelo governo brasileiro para o estabelecimento de políticas públicas."_ - [Coordenação Geral de Observação da Terra INPE](#http://www.obt.inpe.br/OBT/assuntos/programas/amazonia/prodes#:~:text=O%20projeto%20PRODES%20realiza%20o,o%20estabelecimento%20de%20pol%C3%ADticas%20p%C3%BAblicas.)

In [18]:
from modules import *



# DATASETS = [pd.read_csv(f'../Data/DesmatamentoMunicipios20{digits}.txt', sep=',', encoding='iso-8859-1') for digits in range(0,18)]
DATASETS = []

for i in range(0,19):
    a = '../Data/2000_2018/DesmatamentoMunicipiosAM2000.txt'
    if i < 10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM200{i}.txt', sep=',', encoding='iso-8859-1'))
    elif i>=10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM20{i}.txt', sep=',', encoding='iso-8859-1'))
     

type(DATASETS[0])


pandas.core.frame.DataFrame

**DATASETS** é uma lista com 19 dataframes com as informações dos arquivos que estão no diretório _`../Data/2000_2018`_

In [19]:
DATASETS[0].sample(3)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2000,Incremento19992000,Floresta2000,Nuvem2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000,Soma
0,1,-2.83490,-57.83095,S 2 50 5.64,O 57 49 51.42,Urucurituba,1304401,AM,2915,82.3,NaN,142.7,0.0,0.0,1689.6,1000.4,100
58,59,-4.15477,-62.08065,S 4 9 17.17,O 62 4 50.34,Anori,1300102,AM,5852,69.5,NaN,4706.9,0.0,0.3,640.6,434.7,100
24,25,-2.57290,-60.37242,S 2 34 22.44,O 60 22 20.71,Manaus,1302603,AM,11473,1096.7,NaN,8825.3,0.0,0.2,3.0,1547.8,100


In [20]:
DATASETS[1].sample(2)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2001,Incremento20002001,Floresta2001,Nuvem2001,NaoObservado2001,NaoFloresta2001,Hidrografia2001,Soma
11,12,-3.08079,-69.06723,S 3 4 50.84,O 69 4 2.03,Santo Antônio do Içá,1303700,AM,12737,125.7,14.4,12004.4,0.0,65.2,90.4,451.3,100
10,11,0.35952,-68.17017,N 0 21 34.27,O 68 10 12.61,São Gabriel da Cachoeira,1303809,AM,112476,1110.7,67.2,109021.3,1.5,370.6,489.6,1482.3,100


**MunicipalAdjascencyDict** é um dicionário contendo como chave o nome de uma cidade e como valor os municípios de adjascência da mesma

A referência de Ajascência municipal segue o modelo de mapa proposto pelo [Mapa do Escolar feito pelo IBGE](#https://geoftp.ibge.gov.br/produtos_educacionais/mapas_tematicos/mapas_do_brasil/mapas_estaduais/politico/amazonas.jpg) modificado em 2016, segue o exemplo:


In [21]:
MunicipalAdjascencyDict = {
   
'Urucurituba' : 
['Boa Vista do Ramos', 'Barreirinha', 'Itapiranga', 'Itacoatiara', 'Parintins', 'Urucara', 'Silves', 'Nhamunda'],

'Urucara' : 
['Nhamunda', 'Urucurituba', 'Itapiranga', 'Sao Sebastiao do Uatuma', 'Presidente Figueiredo'],

'Uarini' : 
['Maraa','Fonte Boa','Jurua', 'Alvaraes','Tefe'],

'Tonantins' : 
['Japura', 'Santo Antonio do Ica', 'Fonte Boa', 'Jutai'],

'Tefe' : 
['Tapaua', 'Maraa', 'Carauari', 'Jurua', 'Alvaraes', 'Uarini'],

'Tapaua' : 
['Itamarati', 'Pauini', 'Labrea', 'Canutama', 'Carauari', 'Tefe', 'Coari', 'Anori', 'Beruri', 'Manicore', 'Humaita'],

'Tabatinga' : 
['Benjamin Constant', 'Santo Antonio do Ica', 'Sao Paulo de Olivenca'],

'Silves' : 
[ 'Urucurituba', 'Itacoatiara', 'Itapiranga'],

'Sao Sebastiao do Uatuma' : 
['Urucara', 'Presidente Figueiredo', 'Itapiranga'],

'Sao Paulo de Olivenca' : 
['Benjamin Constant', 'Jutai', 'Tabatinga', 'Santo Antonio do Ica', 'Amatura'],

'Sao Gabriel da Cachoeira' : 
['Japura', 'Santa Isabel do Rio Negro'],

'Santo Antonio do Ica' : 
['Sao Paulo de Olivenca', 'Tabatinga', 'Amatura' , 'Tonantins', 'Jutai', 'Japura'],

'Santa Isabel do Rio Negro' : 
['Sao Gabriel da Cachoeira', 'Japura', 'Maraa', 'Barcelos'],

'Rio Preto da Eva' : 
['Itacoatiara', 'Manaus', 'Itapiranga', 'Presidente Figueiredo'],

'Presidente Figueiredo' : 
['Rio Preto da Eva', 'Manaus', 'Novo Airao', 'Itapiranga', 'Sao Sebastiao do Uatuma', 'Urucara'],

'Pauini' : #15
['Boca do Acre', 'Envira', 'Itamarati', 'Eirunepe','Tapaua', 'Labrea'],

'Parintins' : #16
['Nhamunda', 'Urucurituba', 'Barreirinha'],

'Novo Aripuana' : 
['Manicore', 'Apui', 'Borba'],

'Novo Airao' :  #18
['Barcelos', 'Codajas', 'Caapiranga', 'Manacapuru', 'Manaus', 'Presidente Figueiredo'],

'Nova Olinda do Norte' : 
['Borba', 'Maues', 'Autazes', 'Itacoatiara'],

'Nhamunda' :  
['Urucara', 'Urucurituba', 'Parintins', 'Maues'],

'Maues' : #21
['Apui', 'Borba', 'Itacoatiara', 'Boa Vista do Ramos', 'Barreirinha', 'Nova Olinda do Norte'],

'Maraa' : 
['Fonte Boa', 'Japura', 'Santa Isabel do Rio Negro', 'Barcelos', 'Codajas', 'Coari', 'Tefe', 'Uarini'],

'Manicore' : 
['Humaita', 'Tapaua', 'Beruri', 'Borba','Novo Aripuana'],

'Manaus' : 
['Manacapuru','Itacoatiara','Careiro', 'Careiro da Varzea', 'Iranduba', 'Manacapuru', 'Novo Airao', 'Presidente Figueiredo', 'Rio Preto da Eva'],

'Manaquiri' : 
['Manacapuru', 'Borba', 'Beruri', 'Iranduba', 'Manaus', 'Careiro'],

'Manacapuru' : 
['Novo Airao', 'Caapiranga', 'Anama', 'Anori', 'Beruri', 'Manaquiri', 'Iranduba', 'Manaus'],

'Labrea' : 
['Boca do Acre', 'Pauini', 'Itamarati', 'Tapaua', 'Canutama'],

'Jutai' :  
['Itamarati', 'Eirunepe', 'Benjamin Constant', 'Sao Paulo de Olivenca','Amatura','Santo Antonio do Ica', 'Tonantins', 'Fonte Boa', 'Jurua', 'Carauari'],

'Jurua' :  
['Carauari', 'Jutai', 'Fonte Boa', 'Uarini', 'Alvaraes', 'Tefe'],

'Japura' : 
['Santo Antonio do Ica','Tonantins','Fonte Boa', 'Maraa','Santa Isabel do Rio Negro','Sao Gabriel da Cachoeira'],

'Itapiranga' :  
['Presidente Figueiredo','Sao Sebastiao do Uatuma','Urucara','Urucurituba', 'Silves', 'Rio Preto da Eva', 'Itacoatiara'],

'Itamarati' : 
['Envira', 'Eirunepe', 'Jutai', 'Carauari', 'Tapaua','Labrea','Pauini'],

'Itacoatiara' : 
['Autazes', 'Careiro da Varzea', 'Maues', 'Rio Preto da Eva', 'Nova Olinda do Norte', 'Manaus', 'Silves', 'Itapiranga', 'Urucurituba', 'Boa Vista do Ramos'],

'Iranduba' :  
['Novo Airao','Careiro', 'Careiro da Varzea', 'Manacapuru', 'Manaquiri', 'Manaus'],

'Ipixuna' : 
['Eirunepe', 'Benjamin Constant', 'Atalaia do Norte', 'Guajara'],

'Humaita' : 
['Canutama', 'Tapaua', 'Manicore'],

'Guajara' : 
['Atalaia do Norte', 'Ipixuna'],

'Fonte Boa' : 
['Jutai', 'Jurua', 'Tonantins', 'Japura', 'Maraa', 'Uarini'],

'Envira' : 
['Pauini', 'Itamarati', 'Eirunepe'],

'Eirunepe' : 
['Benjamin Constant', 'Ipixuna', 'Envira', 'Itamarati', 'Jutai','Pauini'],

'Codajas' : 
['Anama','Anori','Coari','Maraa','Barcelos','Novo Airao','Caapiranga'],

'Coari' : 
['Maraa', 'Codajas', 'Tefe', 'Tapaua', 'Anori','Barcelos'],

'Careiro da Varzea' : 
['Iranduba', 'Careiro', 'Manaus', 'Itacoatiara', 'Autazes'],

'Careiro' : 
['Iranduba', 'Manacapuru', 'Manaquiri', 'Borba', 'Autazes', 'Careiro da Varzea', 'Manaus'],

'Carauari' : 
['Tefe', 'Alvaraes', 'Jurua', 'Jutai', 'Itamarati', 'Tapaua'],

'Canutama' :  
['Labrea', 'Tapaua', 'Humaita'],

'Caapiranga' : 
['Anama', 'Manacapuru', 'Novo Airao', 'Codajas'],

'Borba' : 
['Maues', 'Nova Olinda do Norte', 'Autazes', 'Careiro', 'Manaquiri', 'Beruri', 'Manicore', 'Novo Aripuana', 'Apui'],

'Boca do Acre' : 
['Pauini', 'Labrea'],

'Boa Vista do Ramos' : 
['Maues','Barreirinha', 'Urucurituba', 'Itacoatiara'],

'Beruri' : 
['Manicore', 'Tapaua', 'Anori', 'Anama', 'Manacapuru', 'Manaquiri'],

'Benjamin Constant' : 
['Atalaia do Norte', 'Ipixuna', 'Eirunepe', 'Jutai', 'Sao Paulo de Olivenca', 'Tabatinga'],

'Barreirinha' : 
['Maues', 'Boa Vista do Ramos', 'Urucurituba', 'Parintins'],

'Barcelos' : 
['Santa Isabel do Rio Negro', 'Novo Airao', 'Maraa', 'Codajas', 'Coari'],

'Autazes' : 
['Manaus', 'Careiro', 'Careiro da Varzea', 'Borba', 'Itacoatiara', 'Nova Olinda do Norte'],

'Atalaia do Norte' :  
['Benjamin Constant', 'Guajara','Ipixuna'],

'Apui' : 
['Novo Aripuana', 'Borba', 'Maues'],

'Anori' : 
['Beruri', 'Tapaua', 'Coari', 'Codajas', 'Anama'],

'Anama' :  
['Manacapuru', 'Beruri', 'Anori','Codajas','Caapiranga'],

'Amatura' : 
['Sao Paulo de Olivenca', 'Jutai', 'Santo Antonio do Ica'],

'Alvaraes' : 
['Tefe', 'Carauari', 'Jurua', 'Uarini']

}


# Limpeza e Inserção de Dados:

A remoção de caracteres e de colunas não utilizadas facilitará o trabalho com as informações essênciais. Além disso a normalização dos dados e estabelecimento de adjascência entre municípios será essêncial para análise destes e modelagem dos grafos.

### Removendo colunas irrelevantes:

In [22]:
print(DATASETS[0].columns)

Index(['Nr', 'Lat', 'Long', 'Latgms', 'Longms', 'Municipio', 'CodIbge',
       'Estado', 'AreaKm2', 'Desmatado2000', 'Incremento19992000',
       'Floresta2000', 'Nuvem2000', 'NaoObservado2000', 'NaoFloresta2000',
       'Hidrografia2000', 'Soma '],
      dtype='object')


In [23]:
DATASETS[0].drop(columns = ['Incremento19992000'], inplace = True)

In [24]:

DATASETS[0]

for index, value in enumerate(DATASETS):
    if index < 10:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem200{index}'], inplace = True)
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Soma '], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)
        value.drop(columns = [f'Floresta200{index}'], inplace = True)
        value.drop(columns = [f'NaoObservado200{index}'], inplace = True)
        value.drop(columns = [f'NaoFloresta200{index}'], inplace = True)
        value.drop(columns = [f'Hidrografia200{index}'], inplace = True)
        
    else:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem20{index}'], inplace = True)
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)
        value.drop(columns = [f'Floresta20{index}'], inplace = True)
        value.drop(columns = [f'NaoObservado20{index}'], inplace = True)
        value.drop(columns = [f'NaoFloresta20{index}'], inplace = True)
        value.drop(columns = [f'Hidrografia20{index}'], inplace = True)

DATASETS[0].sample(2)

,Lat,Long,Municipio,AreaKm2,Desmatado2000
56,-5.67098,-71.72825,Atalaia do Norte,80005,196.1
21,-5.24148,-58.19462,Maués,40090,1129.0


### Tirando acento dos caracteres

In [25]:
#Criando método para retirar os acentos das palavras dentro deste dataset:
def substituir_caracteres(columns):  
  columns = columns.replace('Á','A').replace('Â','A').replace('Ã','A').replace('À','A').replace('É','E').replace('Ê','E').replace('È','E').replace('Í','I').replace('Ì','I').replace('Î','I').replace('Ó','O').replace('Ò','O').replace('Õ','O').replace('Ô','O').replace('Ú','U').replace('Ù','U').replace('Û','U').replace('Ç','C').replace('á','a').replace('â','a').replace('ã','a').replace('à','a').replace('é','e').replace('ê','e').replace('è','e').replace('í','i').replace('ì','i').replace('î','i').replace('ó','o').replace('ò','o').replace('õ','o').replace('ô','o').replace('ú','u').replace('ù','u').replace('û','u').replace('ç','c').replace('-', ' ').replace('.','')
  return columns

for index,dataset in enumerate(DATASETS):
  DATASETS[index]['Municipio'] = DATASETS[index]['Municipio'].apply(substituir_caracteres)

DATASETS[0]['Municipio'].sample(15)

61                 Alvaraes
29                    Jurua
56         Atalaia do Norte
5                    Tapaua
16                Parintins
13         Rio Preto da Eva
32                Itamarati
58                    Anori
34                 Iranduba
14    Presidente Figueiredo
52        Benjamin Constant
3                 Tonantins
19     Nova Olinda do Norte
20                 Nhamunda
24                   Manaus
Name: Municipio, dtype: object

### Criando coluna de fronteira entre municípios para todos os datasets

In [26]:

MunicipalAdjascencyList = [x for x in MunicipalAdjascencyDict.values()]

for i in DATASETS:
    i['Fronteiras'] = MunicipalAdjascencyList

# DATASETS[0].head(2)
print('Boa Vista do Ramos ' in DATASETS[0]['Municipio'])
print('Boa Vista do Ramos' in DATASETS[0]['Municipio'])
DATASETS[0][49:51]
print(DATASETS[0]['Municipio'][50] == 'Boa Vista do Ramos')
print('Boa Vista do Ramos' in DATASETS[0]['Municipio'].unique())


False
False
True
True


### Estabelecendo média de desmatamento:

In [27]:
# mean_column = []

mean = DATASETS[0]['Desmatado2000'].mean()

# print(mean)
for index, dataframe in enumerate(DATASETS):
    mean_column = []
    if index < 10:
        mean = dataframe[f'Desmatado200{index}'].mean()
        for i in dataframe[f'Desmatado200{index}']:
            mean_column.append(True) if i > mean else mean_column.append(False)
    else:
        mean = dataframe[f'Desmatado20{index}'].mean()
        for i in dataframe[f'Desmatado20{index}']:
            mean_column.append(True) if i > mean else mean_column.append(False)

    DATASETS[index]['MaiorQueAMédia'] = mean_column

# exit()

        
#    if DATASETS[0]['Desmatado2000'].mean() < 

# Exportando dados em csv para análise:

A análise dos dado junto à modelagem do grafo são realizados em um arquivo próprio, sendo este exclusivamente utilizado para limpeza dos dados. Os arquivos com dados tratados estão na pasta CleanData2000_2018.

In [28]:

for index, dataframe in enumerate(DATASETS):
        path_to_save = r'~/Área de trabalho/ALL/Desktop/CEFET2/AEDS2/Atividades e Trabalhos/Desenvolvimento/DataAnalysis/DesmatamentoNoAmazonas/Data/CleanData2000_2018/'
        if index < 10:
                dataframe.to_csv(path_to_save+f'Cleaned200{index}.csv' , index = False, encoding='utf-8')
        else:
                dataframe.to_csv(path_to_save+f'Cleaned20{index}.csv', index = False, encoding='utf-8')